In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Data Loading and Preprocessing
class RoadConditionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = cv2.imread(self.images[idx])
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load data
image_paths = ['data/train/image1.jpg', 'data/train/image2.jpg']  # sample paths
labels = [0, 1]  # 0 for smooth, 1 for pothole/crack
train_images, val_images, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2)

train_dataset = RoadConditionDataset(train_images, train_labels, transform=transform)
val_dataset = RoadConditionDataset(val_images, val_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Model Definition
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classification

# Training Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
y_true, y_pred = [], []
for images, labels in val_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    y_true.extend(labels)
    y_pred.extend(predicted)
print(classification_report(y_true, y_pred, target_names=['Smooth', 'Pothole/Crack']))
